In [2]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

In [ ]:
import requests
import base64
import json

In [211]:
from dotenv import load_dotenv
import os

load_dotenv()

subscription_id = os.getenv("SUBSCRIPTION_ID")
resource_group = os.getenv("RESOURCE_GROUP")
workspace = os.getenv("WORKSPACE")

In [4]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [132]:
# Define an endpoint name
endpoint_name = "solar_panel_classify_yusa"

# Example way to define a random name
import datetime

endpoint_name = "endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

DESCRIPTION = "Endpoint to classify solar panel condition into 6 types"

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name, description=DESCRIPTION, auth_mode="key"
)

## Register Model and Environment to AML

In [149]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="../model/best_model.keras",
    type=AssetTypes.CUSTOM_MODEL,
    name="solar_panel_classify_keras",
    description="Model created from local file.",
)
ml_client.models.create_or_update(file_model)

Uploading best_model.keras (< 1 MB): 0.00B [00:00, ?B/s] (< 1 MB): 100%|██████████| 59.0M/59.0M [00:02<00:00, 29.4MB/s] (< 1 MB): 100%|██████████| 59.0M/59.0M [00:02<00:00, 29.3MB/s]




Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'solar_panel_classify_keras', 'description': 'Model created from local file.', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/57c03595-b2a6-45d7-9dc6-7b0da21ff04a/resourceGroups/ml-deploy/providers/Microsoft.MachineLearningServices/workspaces/yusa-aml/models/solar_panel_classify_keras/versions/1', 'Resource__source_path': '', 'base_path': '/Users/yusali/dev/solar-panel-detection/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x160c8d150>, 'serialize': <msrest.serialization.Serializer object at 0x1609a0ed0>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/57c03595-b2a6-45d7-9dc6-7b0da21ff04a/resourceGroups/ml-deploy/workspaces/yusa-aml/datastores/workspaceblobstore/paths/LocalUpload/95c0b87edd39180a4b8ae80075edeb7a/best_model.keras', 'datastore': 

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="../../model-1/environment/conda.yaml",
    name="TF-cv-py311-env",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

### Define **local** Model and Environemnt for local deployment testing

In [94]:
# model = Model(path="../model-1/model/sklearn_regression_model.pkl")
model_local = Model(path="../best_model.onnx")

env_local = Environment(
    conda_file="../environment_tf_py311.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    name="tf-cv-py3111",
    description="Tensorflow cv classification environment with python 3.11",
)

In [ ]:
blue_deployment_local = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model_local,
    environment=env_local,
    code_configuration=CodeConfiguration(code="../src", scoring_script="score.py"),
    instance_type="Standard_D2as_v4",
    instance_count=1,
)

### Define **Online** Model and Environment from AML registry

In [154]:

model = ml_client.models.get(name="solar_panel_classify_keras", version="1")

# model = Model(path="../sklearn_regression_model.pkl")
env = ml_client.environments.get(name="tf-cv-py3111", version="1")

## Instantiate deployment and update env, endpint and deployment

In [ ]:
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(code="../src", scoring_script="score.py"),
    instance_type="Standard_D2as_v4",
    instance_count=1,
)

In [95]:
ml_client.environments.create_or_update(env)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'tf-cv-py3111', 'description': 'Tensorflow cv classification environment with python 3.11', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/57c03595-b2a6-45d7-9dc6-7b0da21ff04a/resourceGroups/ml-deploy/providers/Microsoft.MachineLearningServices/workspaces/yusa-aml/environments/tf-cv-py3111/versions/1', 'Resource__source_path': '', 'base_path': '/Users/yusali/dev/solar-panel-detection/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x125cc3290>, 'serialize': <msrest.serialization.Serializer object at 0x125f20e90>, 'version': '1', 'conda_file': {'channels': ['conda-forge', 'defaults'], 'dependencies': ['python=3.10', 'tensorflow', 'k

In [69]:
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

Updating local endpoint (endpt-07061032887718) Done (0m 0s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:5001/score', 'openapi_uri': None, 'name': 'endpt-07061032887718', 'description': 'this is a sample endpoint', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/Users/yusali/.azureml/inferencing/endpt-07061032887718'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x1375eb590>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [ ]:
ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=True
)

In [82]:
endpoint = ml_client.online_endpoints.get(name=endpoint_name, local=True)
url = endpoint.scoring_uri

print(f"Endpoint URL: {url}")

## Local inferencing

In [90]:
from PIL import Image
import matplotlib.pyplot as plt


# img_path = "../data/Electrical-damage/Electrical (2).JPG"
img_path = "../data/Physical-Damage/Physical (11).jpg"
img = Image.open(img_path)

In [ ]:
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
# Read the image and encode it in Base64
with open(img_path, "rb") as img_file:
    img_base64 = base64.b64encode(img_file.read()).decode("utf-8")

    # Create the payload
    payload = json.dumps({"image": img_base64})

    ml_client.online_endpoints.invoke(
        endpoint_name=endpoint_name,
        # request_file=payload,
        input_data={"image": img_base64},
        local=True,
    )

### Prepare the json file input for inference with onlineendpoint.invoke() method

In [200]:
import os
import json

# Specify the path to the inference_examples folder
inference_examples_folder = "../inference_examples"

# Check if the folder exists
if not os.path.exists(inference_examples_folder):
    print("Error: The 'inference_examples' folder does not exist.")
    os.makedirs(inference_examples_folder)  # Create the folder if it doesn't exist
    print(f"{inference_examples_folder} is created.")
else:
    # Continue with the code
    print(f"{inference_examples_folder} exists.")

    # payload = json.dumps({"image": img_base64})

    with open(os.path.join(inference_examples_folder, "image_payload.json"), "w") as f:
        # json.dump(payload, f)
        json.dump({"image": img_base64}, f)
        print("Payload is saved.")

../inference_examples exists.
Payload is saved.


In [202]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file="../inference_examples/image_payload.json",
    # input_data={"image": img_base64},
)

'"{\\"predicted_class\\": \\"Physical-Damage\\"}"'

### Inference with request.post() using b64encode

TODO: Add the other aaproaches and compare their pros and cons

In [206]:
# Read the image and encode it in Base64
with open(img_path, "rb") as img_file:
    img_base64 = base64.b64encode(img_file.read()).decode("utf-8")

# Create the payload
payload = json.dumps({"image": img_base64})

# Send the request
response = requests.post(
    url, data=payload, headers={"Content-Type": "application/json"}
)

# Print the response
print(response.json())

{"predicted_class": "Physical-Damage"}


In [210]:
def get_prediction(img_path, url):
    with open(img_path, "rb") as img_file:
        img_base64 = base64.b64encode(img_file.read()).decode("utf-8")

    payload = json.dumps({"image": img_base64})

    api_key = os.getenv("API_KEY")
    headers = {
    "Content-Type": "application/json",
    "Authorization": ("Bearer " + api_key),
    "azureml-model-deployment": "blue",
}

    response = requests.post(url, data=payload, headers=headers)
    return response.json()

### Failed inferencing methods, Bytes, multi-part file

In [207]:
# Send the request
response = requests.post(
    url, data=img
)
# Print the response
print(response.json())

TypeError: Image.seek() takes 2 positional arguments but 3 were given

In [208]:
# Send the request with the image as multipart form data
with open(img_path, "rb") as img_file:
    files = {"file": img_file}
    response = requests.post(
        url, files=files, headers={"Content-Type": "multipart/form-data"}
    )

# Send the request with the image as multipart form data
# with open(img_path, "rb") as img_file:
#     files = {"file": img_file}

# response = requests.post(url, files=img, headers={"Content-Type": "multipart/form-data"})

# Print the response
print(response.json())

{'message': "Input cannot be decoded as UTF-8: 'utf-8' codec can't decode byte 0xff in position 113: invalid start byte"}


In [63]:
# Send the request with the image as binary data
with open(img_path, "rb") as img_file:
    img_data = img_file.read()
    headers = {"Content-Type": "application/octet-stream"}
    # files = {"image": ("image.jpg", img_data)}
response = requests.post(url, 
                            data=img_data, 
                            headers=headers,
                            # files=files
                        )

# Print the response
print(response.json())

{'message': "Input cannot be decoded as UTF-8: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte"}


## Deploy on Azure, there's lag between each steps, be patient

In [145]:
ml_client.online_endpoints.begin_create_or_update(endpoint)

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


In [155]:
ml_client.online_deployments.begin_create_or_update(
    blue_deployment, 
)

Check: endpoint endpt-07071109594563 exists
Uploading src (0.02 MBs): 100%|██████████| 19659/19659 [00:00<00:00, 116827.20it/s]




...........................................

In [156]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint)

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


In [157]:
online_deployment = ml_client.online_endpoints.get(name=endpoint_name)

In [158]:
online_url = online_deployment.scoring_uri
print(f"Online URL: {online_url}")

Online URL: https://endpt-07071109594563.eastus2.inference.ml.azure.com/score


In [159]:
for endpoint in ml_client.online_endpoints.list():
    print(endpoint.name)

endpt-07071109594563


In [160]:
print("Kind\tLocation\tName")
print("-------\t----------\t------------------------")
for endpoint in ml_client.online_endpoints.list():
    print(f"{endpoint.kind}\t{endpoint.location}\t{endpoint.name}")

Kind	Location	Name
-------	----------	------------------------
Managed	eastus2	endpt-07071109594563


In [212]:
get_prediction(img_path, online_url)

'{"predicted_class": "Physical-Damage"}'

In [214]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file="../inference_examples/image_payload.json",
    # input_data={"image": img_base64},
)

'"{\\"predicted_class\\": \\"Physical-Damage\\"}"'

## Clean Azure deloyment

In [215]:
ml_client.online_endpoints.begin_delete(name=endpoint_name)

.....